### patsy, statsmodel에 관해 간략하게

모델을 개발하는 과정에서 중요한 단계가 feature engineering인데 원시 데이터셋으로부터 모델링에서 유용할 수 있는 정보를 추출하는 변환이나 분석과정을 일컫는다.

In [1]:
import pandas as pd
import numpy as np

# category를 더미 값으로

In [2]:
data = pd.DataFrame({'x0':[1,2,3,4,5],'x1':[0.01,-0.01,0.25,-4.1,0.],'y':[-1.5,0,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
# Categories 는 R에서의 factor와 비슷하다고 보면 된다.
# 세부적인 차이점은 있지만 기능은 비슷함

data['category'] = pd.Categorical(['a','b','a','a','b'],categories=['a','b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [4]:
dummies = pd.get_dummies(data['category'])
data_dropped = data.drop('category',axis=1)
dummies

category,a,b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [5]:
data_with_dummies = data_dropped.join(dummies)
data_with_dummies

,x0,x1,y,a,b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# Patsy를 이용해 모델 생성하기

In [6]:
data.drop('category',axis=1,inplace=True)
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [7]:
import patsy
y,X = patsy.dmatrices('y~x0+x1',data)

In [8]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [12]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

### DesignMatrix 타입인데 asarray 메서드로 ndarray로 변환시킬 수 있다.

In [13]:
type(X)

patsy.design_info.DesignMatrix

In [16]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [17]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [18]:
# intercept는 0을 더해서 제거할 수 있다.
patsy.dmatrices('y~x0+x1+0',data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

## np.linalg의 메서드를 이용해 최소자승회귀분석을 수행할 수 있다.
참고)<br>
- diag : 정사각행렬의 대각/비대각 원소를 1차원 배열로 반환 or 1차원 배열을 대각선 원소로 하고 나머지는 0으로 채운 단위행렬을 반환
- dot : 행렬곱
- trace : 대각선 원소의 합
- det : 행렬식
- eig : 정사각 행렬의 고유값, 고유벡터
- inv : 정사각 행렬의 역행력
- qr : QR 분해
- svd : 특잇값 분해(SVD)
<br>

- solve : A가 정사각 행렬일 때 Ax = b를 만족하는 x를 구한다.
- lstsq : Ax=b를 만족하는 최소제곱해를 구한다.

In [25]:
coef, residuals, rank, s = np.linalg.lstsq(X,y)
# s는 X의 singular values

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [26]:
# 계수(순서대로 절편, x0, x1)
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [27]:
# 오차
residuals

array([19.63791494])

In [29]:
rank

3

In [30]:
s

array([8.03737688, 3.38335321, 0.90895207])

In [40]:
# 참고) squeeze : Remove single-dimensional entries from the shape of an array.
print(coef.shape)
print(coef.squeeze().shape)

(3, 1)
(3,)


### scikit-learn의 LinearRegression과 같은 계수

In [31]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(data.iloc[:,:-1],data.iloc[:,-1])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [34]:
print(lr.coef_)
print(lr.intercept_)

[-0.07910564 -0.26546384]
0.3129097576804629


## Patsy 용법으로 데이터 변환하기

In [41]:
# dmatrices 내에 여러 함수들을 결합해서 계수를 구할 수 있다.

y,X = patsy.dmatrices('y~x0+np.log(np.abs(x1)+1)',data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [93]:
# 다음은 Patsy의 내장함수들(표준화와 센터링(평균값을 뺌))
y,X = patsy.dmatrices('y~standardize(x0)+center(x1)',data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [94]:
# 바로 직전에 수행한 선형회귀를 기억하고 있음, 
# centering이나 standardize에서 쓰이는 평균, 표준편차도 new_data 기준이 아니라 위에서 쓰인 data 기준임

new_data = pd.DataFrame({
    'x0':[6,7,8,9],
    'x1':[3.1,-0.5,0,2.3],
    'x2':[5,1,2,3],
    'y':[1,2,3,4]})
new_X = patsy.build_design_matrices([X.design_info],new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [95]:
# Patsy 문법에서 더하기(+) 기호는 덧셈이 아니므로 실제 +를 사용하고 싶다면 I라는 특수 함수로 둘러싸야 한다.
y,X = patsy.dmatrices('y~I(x0+x1)',data = data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [103]:
y,X = patsy.dmatrices('y~x0:x2+x1',data = new_data)
X

DesignMatrix with shape (4, 3)
  Intercept  x0:x2    x1
          1     30   3.1
          1      7  -0.5
          1     16   0.0
          1     27   2.3
  Terms:
    'Intercept' (column 0)
    'x0:x2' (column 1)
    'x1' (column 2)

## 범주형 데이터와 Patsy

Patsy에서 비산술 용법을 사용하면 기본적으로 더미 변수로 변환된다. intercept가 존재한다면 공선성을 피하기 위해 레벨 중 하나는 남겨놓게 된다.

In [104]:
data = pd.DataFrame({
    'key1':['a','a','b','b','a','b','a','b'],
    'key2':[0,1,0,1,0,1,0,0],
    'v1':[1,2,3,4,5,6,7,8],
    'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]
})

y,X = patsy.dmatrices('v2~key1',data)

# 더미변수로 변환
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [105]:
y,X = patsy.dmatrices('v2~key1+0',data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

산술 컬럼은 C 함수를 이용해 범주형으로 해석할 수 있다.

In [106]:
# 0,1 등 수치형이지만 범주형 자료로 해석한다.
y,X = patsy.dmatrices('v2~C(key2)',data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

모델에서 여러 범주형 항을 사용한다면 ANOVA 모델에서처럼 key1:key2 같은 용법도 사용할 수 있으므로 복잡해진다.

In [110]:
data['key2'] = data['key2'].map({0:'zeros',1:'one'})
data

,key1,key2,v1,v2
0,a,zeros,1,-1.0
1,a,one,2,0.0
2,b,zeros,3,2.5
3,b,one,4,-0.5
4,a,zeros,5,4.0
5,b,one,6,-1.2
6,a,zeros,7,0.2
7,b,zeros,8,-1.7


In [112]:
y,X = patsy.dmatrices('v2~key1+key2',data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zeros]
          1          0              1
          1          0              0
          1          1              1
          1          1              0
          1          0              1
          1          1              0
          1          0              1
          1          1              1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [113]:
y,X = patsy.dmatrices('v2~key1+key2+key1:key2',data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zeros]  key1[T.b]:key2[T.zeros]
          1          0              1                        0
          1          0              0                        0
          1          1              1                        1
          1          1              0                        0
          1          0              1                        0
          1          1              0                        0
          1          0              1                        0
          1          1              1                        1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# statsmodels
다음과 같은 모델들을 포함한다.<br>
- 선형모델, 일반 선형모델, robust 선형모델
- 선형 복합효과(Linear Mixed Effects, LME) 모델
- ANOVA 메서드
- 시계열 처리 및 상태 공간 모델
- 일반적률추정밥(GMM, Generalized Method of Moments)

In [114]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [119]:
# 랜덤 데이터 생성
def dnorm(mean, variance, size=1):
    if isinstance(size,int):
        size = size,
    return mean + np.sqrt(variance)*np.random.randn(*size)

In [118]:
# 참고) isinstance
print(isinstance((3,6),tuple))
print(isinstance((3,6),list))

True
False


In [122]:
# np.c_는 세로모양 ndarray를 오른쪽으로 붙이는 것(R의 cbind)
# np.r_는 그 반대(R의 rbind)

np.random.seed(12345)
N=100
X=np.c_[dnorm(0,0.4,N),
       dnorm(0,0.6,N),
       dnorm(0,0.2,N)]
eps = dnorm(0,0.1,N)
beta = [0.1,0.3,0.5]

y = np.dot(X,beta) + eps

In [125]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [126]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [129]:
# intercept 컬럼을 추가
X_model = sm.add_constant(X)
X_model[:5].round(4)

array([[ 1.    , -0.1295, -1.2128,  0.5042],
       [ 1.    ,  0.3029, -0.4357, -0.2542],
       [ 1.    , -0.3285, -0.0253,  0.1384],
       [ 1.    , -0.3515, -0.7196, -0.2582],
       [ 1.    ,  1.2433, -0.3738, -0.5226]])

## sm.OLS 클래스는 최소자승 선형회귀에 피팅할 수 있다.

OLS는 ordinary least squares(최소제곱)<br>
IRLS(iteratively reweighted least squares, 반복재가중 최소제곱)같은 복잡한 선형회귀 모델도 구현 가능

In [144]:
# intercept 항은 없는 상태
model = sm.OLS(y,X)
result = model.fit()
result.params

array([0.17826108, 0.22303962, 0.50095093])

In [145]:
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Wed, 12 Aug 2020   Prob (F-statistic):                    7.44e-12
Time:                        19:11:04   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## y,X가 모두 같은 DataFrame에 들어가 있는 경우

In [133]:
data = pd.DataFrame(X,columns=['col0','col1','col2'])
data['y'] = y
data.head()

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [134]:
# statsmodels의 API와 Patsy 문자열 용법을 사용할 수 있다.(smf의 소문자 ols)

results = smf.ols('y~col0+col1+col2',data=data).fit()

In [137]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [138]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [139]:
results.pvalues

Intercept    3.433928e-01
col0         1.274519e-03
col1         4.745425e-06
col2         8.817014e-09
dtype: float64

In [141]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     24.68
Date:                Wed, 12 Aug 2020   Prob (F-statistic):           6.37e-12
Time:                        19:10:24   Log-Likelihood:                -33.835
No. Observations:                 100   AIC:                             75.67
Df Residuals:                      96   BIC:                             86.09
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0336      0.035      0.952      0.3

In [154]:
# 예측값 (target값을 포함해도 그만, 안해도 그만)
results.predict(data.iloc[:,:-1][:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

In [155]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

In [149]:
# 실제값
data[:5]['y']

0    0.427863
1   -0.673480
2   -0.090878
3   -0.489494
4   -0.128941
Name: y, dtype: float64

## 시계열 처리 예측

In [157]:
# 데이터
init_x = 4
import random
values = [init_x,init_x]
N=1000

b0 = 0.8
b1 = -0.4
noise = dnorm(1,0.1,N)

for i in range(N):
    new_x = values[-1]*b0 + values[-2]*b1 + noise[i]
    values.append(new_x)

위 데이터는 인자가 0.8, -0.4인 AR(2)(이동평균모형) 구조다. AR 모델을 피팅할 때는 포함시켜야할 지연 항을 얼마나 두어야하는지 알지 못하므로 적당히 큰 값으로 모델을 피팅한다.

In [160]:
maxlags = 5

# 시계열 분석을 위한 메서드
model = sm.tsa.AR(values)

results = model.fit(maxlags)

results.params

array([ 1.00875743,  0.78476827, -0.40854388, -0.0134874 ,  0.01773327,
        0.01063382])